![piggy_bank](piggy_bank.jpeg)

<br>

Personal loans are a lucrative revenue stream for banks. The typical interest rate of a two year loan in the United Kingdom is [around 10%](https://www.experian.com/blogs/ask-experian/whats-a-good-interest-rate-for-a-personal-loan/). This might not sound like a lot, but in September 2022 alone UK consumers borrowed [around £1.5 billion](https://www.ukfinance.org.uk/system/files/2022-12/Household%20Finance%20Review%202022%20Q3-%20Final.pdf), which would mean approximately £300 million in interest generated by banks over two years!

You have been asked to work with a bank to clean and store the data they collected as part of a recent marketing campaign, which aimed to get customers to take out a personal loan. They plan to conduct more marketing campaigns going forward so would like you to set up a PostgreSQL database to store this campaign's data, designing the schema in a way that would allow data from future campaigns to be easily imported. 

They have supplied you with a csv file called `"bank_marketing.csv"`, which you will need to clean, reformat, and split, in order to save separate files based on the tables you will create. It is recommended to use `pandas` for these tasks.

Lastly, you will write the SQL code that the bank can execute to create the tables and populate with the data from the csv files. As the bank are quite strict about their security, you'll provide the database design script as a `.sql` file that they can then run. 

You have been asked to design a database that will have three tables:

## client

| column | data type | description |
|--------|-----------|-------------|
| `id` | `serial` | Client ID - primary key |
| `age` | `integer` | Client's age in years |
| `job` | `text` | Client's type of job |
| `marital` | `text` | Client's marital status |
| `education` | `text` | Client's level of education |
| `credit_default` | `boolean` | Whether the client's credit is in default |
| `housing` | `boolean` | Whether the client has an existing housing loan (mortgage) |
| `loan` | `boolean` | Whether the client has an existing personal loan |

<br>

## campaign

| column | data type | description |
|--------|-----------|-------------|
| `campaign_id` | `serial` | Campaign ID - primary key |
| `client_id` | `serial` | Client ID - references `id` in the `client` table |
| `number_contacts` | `integer` | Number of contact attempts to the client in the current campaign |
| `contact_duration` | `integer` | Last contact duration in seconds |
| `pdays` | `integer` | Number of days since contact in previous campaign (`999` = not previously contacted) |
| `previous_campaign_contacts` | `integer` | Number of contact attempts to the client in the previous campaign |
| `previous_outcome` | `boolean` | Outcome of the previous campaign |
| `campaign_outcome` | `boolean` | Outcome of the current campaign |
| `last_contact_date` | `date` | Last date the client was contacted |

<br>

## economics

| column | data type | description |
|--------|-----------|-------------|
| `client_id` | `serial` | Client ID - references `id` in the `client` table |
| `emp_var_rate` | `float` | Employment variation rate (quarterly indicator) |
| `cons_price_idx` | `float` | Consumer price index (monthly indicator) |
| `euribor_three_months` | `float` | Euro Interbank Offered Rate (euribor) three month rate (daily indicator) |
| `number_employed` | `float` | Number of employees (quarterly indicator)| 

In [1]:
import pandas as pd
import numpy as np

In [54]:
bank_marketing_df = pd.read_csv('bank_marketing.csv')

In [5]:
bank_marketing_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 22 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   client_id       41188 non-null  int64  
 1   age             41188 non-null  int64  
 2   job             41188 non-null  object 
 3   marital         41188 non-null  object 
 4   education       41188 non-null  object 
 5   credit_default  41188 non-null  object 
 6   housing         41188 non-null  object 
 7   loan            41188 non-null  object 
 8   contact         41188 non-null  object 
 9   month           41188 non-null  object 
 10  day             41188 non-null  int64  
 11  duration        41188 non-null  int64  
 12  campaign        41188 non-null  int64  
 13  pdays           41188 non-null  int64  
 14  previous        41188 non-null  int64  
 15  poutcome        41188 non-null  object 
 16  emp_var_rate    41188 non-null  float64
 17  cons_price_idx  41188 non-null 

In [6]:
bank_marketing_df.head()

,client_id,age,job,marital,education,credit_default,housing,loan,contact,month,...,campaign,pdays,previous,poutcome,emp_var_rate,cons_price_idx,cons_conf_idx,euribor3m,nr_employed,y
0,0,56,housemaid,married,basic.4y,no,no,no,telephone,may,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,1,57,services,married,high.school,unknown,no,no,telephone,may,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,2,37,services,married,high.school,no,yes,no,telephone,may,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,3,40,admin.,married,basic.6y,no,no,no,telephone,may,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,4,56,services,married,high.school,no,no,yes,telephone,may,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


In [71]:
client_columns = ['client_id','age','job','marital','education','credit_default','housing','loan']
campaign_columns = ['campaign','client_id','contact','duration','pdays','previous','poutcome','month','day','y']
economics_columns = ['client_id','emp_var_rate','cons_price_idx','euribor3m','nr_employed']

In [72]:
client_df = bank_marketing_df[client_columns]
campaign_df = bank_marketing_df[campaign_columns]
economic_df = bank_marketing_df[economics_columns]

In [73]:
client_df.head()

,client_id,age,job,marital,education,credit_default,housing,loan
0,0,56,housemaid,married,basic.4y,no,no,no
1,1,57,services,married,high.school,unknown,no,no
2,2,37,services,married,high.school,no,yes,no
3,3,40,admin.,married,basic.6y,no,no,no
4,4,56,services,married,high.school,no,no,yes


In [74]:
campaign_df.head()

,campaign,client_id,contact,duration,pdays,previous,poutcome,month,day,y
0,1,0,telephone,261,999,0,nonexistent,may,13,no
1,1,1,telephone,149,999,0,nonexistent,may,19,no
2,1,2,telephone,226,999,0,nonexistent,may,23,no
3,1,3,telephone,151,999,0,nonexistent,may,27,no
4,1,4,telephone,307,999,0,nonexistent,may,3,no


In [75]:
economic_df.head()

,client_id,emp_var_rate,cons_price_idx,euribor3m,nr_employed
0,0,1.1,93.994,4.857,5191.0
1,1,1.1,93.994,4.857,5191.0
2,2,1.1,93.994,4.857,5191.0
3,3,1.1,93.994,4.857,5191.0
4,4,1.1,93.994,4.857,5191.0


In [76]:
client_columns_rename = {'client_id':'id'}
campaign_columns_rename = {'duration':'contact_duration','previous':'previous_campaign_contacts','y':'campaign_outcome','poutcome':'previous_outcome','campaign':'number_contacts'}
economic_columns_rename = {'euribor3m':'euribor_three_months','nr_employed':'number_employed'}

client_df.rename(columns=client_columns_rename,inplace=True)
campaign_df.rename(columns=campaign_columns_rename,inplace=True)
economic_df.rename(columns=economic_columns_rename,inplace=True)

/var/folders/2k/5bj681bn2996w9njz8q7sf780000gn/T/ipykernel_60356/2162277353.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  client_df.rename(columns=client_columns_rename,inplace=True)
/var/folders/2k/5bj681bn2996w9njz8q7sf780000gn/T/ipykernel_60356/2162277353.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  campaign_df.rename(columns=campaign_columns_rename,inplace=True)
/var/folders/2k/5bj681bn2996w9njz8q7sf780000gn/T/ipykernel_60356/2162277353.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas

In [77]:
client_df.head()

,id,age,job,marital,education,credit_default,housing,loan
0,0,56,housemaid,married,basic.4y,no,no,no
1,1,57,services,married,high.school,unknown,no,no
2,2,37,services,married,high.school,no,yes,no
3,3,40,admin.,married,basic.6y,no,no,no
4,4,56,services,married,high.school,no,no,yes


In [78]:
client_df['education'] = client_df['education'].str.replace('.','_')
client_df.head()

/var/folders/2k/5bj681bn2996w9njz8q7sf780000gn/T/ipykernel_60356/3934316767.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  client_df['education'] = client_df['education'].str.replace('.','_')


,id,age,job,marital,education,credit_default,housing,loan
0,0,56,housemaid,married,basic_4y,no,no,no
1,1,57,services,married,high_school,unknown,no,no
2,2,37,services,married,high_school,no,yes,no
3,3,40,admin.,married,basic_6y,no,no,no
4,4,56,services,married,high_school,no,no,yes


In [79]:
client_df.loc[client_df['education']=='unknown','education'] = np.nan

In [80]:
client_df.loc[:,'job'] = client_df['job'].str.replace('.','')

In [81]:
campaign_df.head()

,number_contacts,client_id,contact,contact_duration,pdays,previous_campaign_contacts,previous_outcome,month,day,campaign_outcome
0,1,0,telephone,261,999,0,nonexistent,may,13,no
1,1,1,telephone,149,999,0,nonexistent,may,19,no
2,1,2,telephone,226,999,0,nonexistent,may,23,no
3,1,3,telephone,151,999,0,nonexistent,may,27,no
4,1,4,telephone,307,999,0,nonexistent,may,3,no


In [83]:
def convert_values(row):
    result = any
    if row['previous_outcome'] == 'nonexistent':
        result = np.nan
    if row['previous_outcome'] == 'failure':
        result = 0
    if row['previous_outcome'] == 'success':
        result = 1
    return result

In [84]:
convert_values_apply = campaign_df.apply(lambda row: convert_values(row),axis=1)
convert_values_apply

0        NaN
1        NaN
2        NaN
3        NaN
4        NaN
        ... 
41183    NaN
41184    NaN
41185    NaN
41186    NaN
41187    0.0
Length: 41188, dtype: float64

In [85]:
campaign_df.loc[:,'previous_outcome'] = convert_values_apply

In [87]:
campaign_df['previous_outcome'].unique()

array([nan, 0.0, 1.0], dtype=object)

In [89]:
campaign_df['campaign_id']=1

/var/folders/2k/5bj681bn2996w9njz8q7sf780000gn/T/ipykernel_60356/1744431624.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  campaign_df['campaign_id']=1


In [91]:
campaign_df.tail()

,number_contacts,client_id,contact,contact_duration,pdays,previous_campaign_contacts,previous_outcome,month,day,campaign_outcome,campaign_id
41183,1,41183,cellular,334,999,0,NaN,nov,30,yes,1
41184,1,41184,cellular,383,999,0,NaN,nov,6,no,1
41185,2,41185,cellular,189,999,0,NaN,nov,24,no,1
41186,1,41186,cellular,442,999,0,NaN,nov,17,yes,1
41187,3,41187,cellular,239,999,1,0.0,nov,23,no,1


In [92]:
campaign_df.loc[:,'year'] = 2022
campaign_df.head()

/var/folders/2k/5bj681bn2996w9njz8q7sf780000gn/T/ipykernel_60356/41279413.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  campaign_df.loc[:,'year'] = 2022


,number_contacts,client_id,contact,contact_duration,pdays,previous_campaign_contacts,previous_outcome,month,day,campaign_outcome,campaign_id,year
0,1,0,telephone,261,999,0,NaN,may,13,no,1,2022
1,1,1,telephone,149,999,0,NaN,may,19,no,1,2022
2,1,2,telephone,226,999,0,NaN,may,23,no,1,2022
3,1,3,telephone,151,999,0,NaN,may,27,no,1,2022
4,1,4,telephone,307,999,0,NaN,may,3,no,1,2022


In [107]:
campaign_df.loc[:,'month'] =  pd.to_datetime(campaign_df.month,format='%b').dt.month

In [110]:
date_columns = ['month', 'day','year']
campaign_df.loc[:,'last_contact_date'] = pd.to_datetime(campaign_df[date_columns]).dt.strftime('%Y-%m-%d')
campaign_df.tail()

,number_contacts,client_id,contact,contact_duration,pdays,previous_campaign_contacts,previous_outcome,month,day,campaign_outcome,campaign_id,year,last_contact_date
41183,1,41183,cellular,334,999,0,NaN,11,30,yes,1,2022,2022-11-30
41184,1,41184,cellular,383,999,0,NaN,11,6,no,1,2022,2022-11-06
41185,2,41185,cellular,189,999,0,NaN,11,24,no,1,2022,2022-11-24
41186,1,41186,cellular,442,999,0,NaN,11,17,yes,1,2022,2022-11-17
41187,3,41187,cellular,239,999,1,0.0,11,23,no,1,2022,2022-11-23


In [114]:
campaign_df.drop(date_columns, inplace=True, axis=1)
campaign_df.head()

/var/folders/2k/5bj681bn2996w9njz8q7sf780000gn/T/ipykernel_60356/1347174307.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  campaign_df.drop(date_columns, inplace=True, axis=1)


,number_contacts,client_id,contact,contact_duration,pdays,previous_campaign_contacts,previous_outcome,campaign_outcome,campaign_id,last_contact_date
0,1,0,telephone,261,999,0,NaN,no,1,2022-05-13
1,1,1,telephone,149,999,0,NaN,no,1,2022-05-19
2,1,2,telephone,226,999,0,NaN,no,1,2022-05-23
3,1,3,telephone,151,999,0,NaN,no,1,2022-05-27
4,1,4,telephone,307,999,0,NaN,no,1,2022-05-03


In [115]:
campaign_df.drop(['contact'],inplace=True,axis=1)
campaign_df.tail()

/var/folders/2k/5bj681bn2996w9njz8q7sf780000gn/T/ipykernel_60356/3178085029.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  campaign_df.drop(['contact'],inplace=True,axis=1)


,number_contacts,client_id,contact_duration,pdays,previous_campaign_contacts,previous_outcome,campaign_outcome,campaign_id,last_contact_date
41183,1,41183,334,999,0,NaN,yes,1,2022-11-30
41184,1,41184,383,999,0,NaN,no,1,2022-11-06
41185,2,41185,189,999,0,NaN,no,1,2022-11-24
41186,1,41186,442,999,0,NaN,yes,1,2022-11-17
41187,3,41187,239,999,1,0.0,no,1,2022-11-23


In [116]:
columns_order = ['campaign_id','client_id','number_contacts','contact_duration','pdays','previous_campaign_contacts','previous_outcome','campaign_outcome','last_contact_date']
campaign_df = campaign_df.reindex(columns=columns_order)
campaign_df.head()

,campaign_id,client_id,number_contacts,contact_duration,pdays,previous_campaign_contacts,previous_outcome,campaign_outcome,last_contact_date
0,1,0,1,261,999,0,NaN,no,2022-05-13
1,1,1,1,149,999,0,NaN,no,2022-05-19
2,1,2,1,226,999,0,NaN,no,2022-05-23
3,1,3,1,151,999,0,NaN,no,2022-05-27
4,1,4,1,307,999,0,NaN,no,2022-05-03


In [117]:
columns_order = ['id','age','job','marital','education','credit_default','housing','loan']
client_df = client_df.reindex(columns=columns_order)
client_df.head()

,id,age,job,marital,education,credit_default,housing,loan
0,0,56,housemaid,married,basic_4y,no,no,no
1,1,57,services,married,high_school,unknown,no,no
2,2,37,services,married,high_school,no,yes,no
3,3,40,admin,married,basic_6y,no,no,no
4,4,56,services,married,high_school,no,no,yes


In [118]:
columns_order=['client_id','emp_var_rate','cons_price_idx','euribor_three_months','number_employed']
economic_df = economic_df.reindex(columns=columns_order)
economic_df.head()

,client_id,emp_var_rate,cons_price_idx,euribor_three_months,number_employed
0,0,1.1,93.994,4.857,5191.0
1,1,1.1,93.994,4.857,5191.0
2,2,1.1,93.994,4.857,5191.0
3,3,1.1,93.994,4.857,5191.0
4,4,1.1,93.994,4.857,5191.0


In [120]:
client_df.to_csv('client.csv',index=False)
economic_df.to_csv('campaign.csv',index=False)
campaign_df.to_csv('economics.csv',index=False)

In [121]:
client_table = """CREATE TABLE client(
    id SERIAL PRIMARY KEY,
    age INT,
    job TEXT,
    marital TEXT,
    education TEXT,
    credit_default BOOL,
    housing BOOL,
    loan BOOL
);
\copy client from 'client.csv' DELIMITER ',' CSV HEADER
"""

In [122]:
campaign_table = """CREATE TABLE campaign(
    campaign_id SERIAL PRIMARY KEY,
    client_id SERIAL REFERENCES client (id),
    number_contacts INT,
    contact_duration INT,
    pdays INT,
    previous_campaign_contacts INT,
    previous_outcome BOOL,
    campaign_outcome BOOL,
    last_contact_date DATE
);
\copy campaign from 'campaign.csv' DELIMITER ',' CSV HEADER
"""

In [123]:
economics_table = """CREATE TABLE economics(
    client_id SERIAL REFERENCES client (id),
    emp_var_rate FLOAT8,
    cons_price_id FLOAT8,
    euribor_three_months FLOAT8,
    number_employed FLOAT8,
    CONSTRAINT client_id FOREIGN KEY(client_id) REFERENCES client(id)
);
\copy economics from 'economics.csv' DELIMITER ',' CSV HEADER
"""